In [6]:
# ./countdown/train.parquet
import pandas as pd

data = pd.read_parquet("./data/train.parquet")
print(len(data))
# Print as df
print(data.loc[0])
print(data.loc[0]["prompt"])
print(data.loc[0]["reward_model"])
print(type(data.loc[0]["extra_info"]["index"]))


32768
target                                                         98
nums                                                 [44, 19, 35]
data_source                                             countdown
prompt          [{'content': 'A conversation between User and ...
ability                                                      math
reward_model    {'ground_truth': {'numbers': [44, 19, 35], 'ta...
extra_info                         {'index': 0, 'split': 'train'}
Name: 0, dtype: object
[{'content': 'A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.\nUser: Using the numbers [44, 19, 35], create an equation that equals 98. You can use basic arithmetic operations (+, -, *, /) and each number can only be used once. Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> (1

In [5]:
import pyarrow.parquet as pq
import json

# Your file path
parquet_file = "data/train.parquet"

# 1. Use pyarrow to read the file schema
schema = pq.read_schema(parquet_file)

# 2. Extract metadata from schema
#    The datasets library stores its information under the 'huggingface' key
metadata = schema.metadata
hf_metadata_key = b"huggingface"  # Note that the key is of bytes type

if hf_metadata_key not in metadata:
    print("Error: Hugging Face metadata not found in the Parquet file.")
    print(
        "This may mean the file was not created by the datasets library, or the metadata has been lost."
    )
    print("Please try method 2 below.")
else:
    # 3. Parse the metadata
    #    The metadata is a JSON string that needs to be decoded and loaded
    hf_info_str = metadata[hf_metadata_key].decode("utf-8")
    hf_info_dict = json.loads(hf_info_str)

    # 4. Extract features definition
    features_dict = hf_info_dict.get("info", {}).get("features", {})

    print("--- Features Metadata Stored in the File ---")
    # Use json.dumps to format the output for better readability
    print(json.dumps(features_dict, indent=2))
    print("-------------------------------------------\n")

    # 5. Recursively search for features with type 'List'
    print("Searching for features with type 'List'...")

    def find_problematic_features(features, path=""):
        for name, definition in features.items():
            current_path = f"{path}.{name}" if path else name
            if isinstance(definition, dict):
                if definition.get("_type") == "List":
                    print(f"==> Found it! Feature '{current_path}' has type 'List'.")
                # Recursively search for nested features
                find_problematic_features(definition, current_path)

    find_problematic_features(features_dict)

--- Features Metadata Stored in the File ---
{
  "target": {
    "dtype": "int64",
    "_type": "Value"
  },
  "nums": {
    "feature": {
      "dtype": "int64",
      "_type": "Value"
    },
    "_type": "List"
  },
  "data_source": {
    "dtype": "string",
    "_type": "Value"
  },
  "prompt": {
    "feature": {
      "content": {
        "dtype": "string",
        "_type": "Value"
      },
      "role": {
        "dtype": "string",
        "_type": "Value"
      }
    },
    "_type": "List"
  },
  "ability": {
    "dtype": "string",
    "_type": "Value"
  },
  "reward_model": {
    "ground_truth": {
      "numbers": {
        "feature": {
          "dtype": "int64",
          "_type": "Value"
        },
        "_type": "List"
      },
      "target": {
        "dtype": "int64",
        "_type": "Value"
      }
    },
    "style": {
      "dtype": "string",
      "_type": "Value"
    }
  },
  "extra_info": {
    "index": {
      "dtype": "int64",
      "_type": "Value"
    },
    "s